In [1]:
import pandas as pd
import gc
from collections import defaultdict
from tqdm import tqdm_notebook as tqdm
gc.collect()

0

In [2]:
sub_rel_dir = "submission_rel2.csv"
sub_is_dir = "submission_is.csv"

In [3]:
df_rel = pd.read_csv(sub_rel_dir)
df_is = pd.read_csv(sub_is_dir)
df_bbox_submission = pd.read_csv("submission.csv")
list_test_imageIds = df_bbox_submission["ImageId"].values
del df_bbox_submission

In [4]:
df = pd.concat([df_rel, df_is])
# df = df_rel

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [5]:
len(df)

3853289

In [6]:
list_imgids = list(df['ImageID'].unique())

In [7]:
len(list_imgids)

54112

In [8]:
dict_pred = defaultdict(str)

In [9]:
for index, row in tqdm(df.iterrows()):
    imgid = row["ImageID"]
    confidence = (row["Confidence1"] + row["Confidence2"])/2
    label1 = row["LabelName1"]
    XMin1, YMin1, XMax1, YMax1 = row["XMin1"], row["YMin1"], row["XMax1"], row["YMax1"] 
    label2 = row["LabelName2"]
    XMin2, YMin2, XMax2, YMax2 = row["XMin2"], row["YMin2"], row["XMax2"], row["YMax2"] 
    relLabel = row["RelationshipLabel"]
    predStr = str(confidence) + " " + str(label1) + " " + \
      str(XMin1) + " "+ str(YMin1) + " " + str(XMax1) + " "+ str(YMax1) + " " + \
      str(label2) + " " + \
      str(XMin2) + " "+ str(YMin2) + " " + str(XMax2) + " "+ str(YMax2) + " " + \
      str(relLabel) + " "
    dict_pred[imgid]+=predStr

In [10]:
final_sub = []
for key, value in tqdm(dict_pred.items()):
    dict_res = {
      "ImageId": key,
      "PredictionString": value,
    }
    final_sub.append(dict_res)

In [11]:
df_submit = pd.DataFrame(final_sub)

In [12]:
list_predicted_imgs = df_submit.ImageId.values
main_list = list(set(list_test_imageIds) - set(list_predicted_imgs))
len(main_list)

45887

In [13]:
append_unpredicted = []
for ml in main_list:
    append_unpredicted.append({
      "ImageId":ml,
      "PredictionString":""
    })
df_append = pd.DataFrame(append_unpredicted)

In [14]:
df_final = pd.concat([df_submit, df_append], ignore_index=True, sort =False)

In [15]:
df_final.head()

,ImageId,PredictionString
0,00000b4dcff7f799,0.2998027503490448 /m/04yx4 0.86089194 0.30551...
1,00001a21632de752,0.45690925419330597 /m/03bt1vf 0.5485642 0.598...
2,0001dd930912683d,0.4464254043996334 /m/04yx4 0.72589636 0.08864...
3,0002f94fe2d2eb9f,0.004397584241814911 /m/04yx4 0.18545882 0.0 0...
4,0003d1c3be9ed3d6,0.42891743453219533 /m/04yx4 0.5961155 0.10516...


In [16]:
len(df_final)

99999

In [17]:
df_final.to_csv("vrd_submission3_combined.csv", index=False)